In [1]:
import pandas as pd
import numpy as np

In [2]:
installs = pd.read_csv('../datos/installs.csv', dtype ={'device_language' : 'str', 'click_hash': object, 'wifi': object, 'trans_id': object})

In [3]:
# No tiene sentido quedarse con el pais, porque es uno solo
#No tiene sentido quedarse con la columna trans_id. El 98% de los valores son Null
#No tiene sentido quedarse con click_hash, el 99% de los valors son null
#No tiene sentido quedarse con event_uuid, el 78% de los valores son null
#No tiene sentido quedarse con kind, el 78% de los valores son null
#No tiene sentido quedarse con wifi, el 48% de los valores son null
#No tiene sentido quedarse con device_brand. El 42% son null
#No tiene sentido quedarse con la columna user_agent El 31% de los valores son Null
#¿Tiene sentido quedarse con la columna session_user_agent? El 3% de los valores son Null
# session_user_agent tiene el 3% de los valores null. Borro esas filas
del installs['trans_id']
del installs['device_countrycode']
del installs['click_hash']
del installs['event_uuid']
del installs['kind']
del installs['wifi']
del installs['device_brand']
del installs['user_agent']
del installs['session_user_agent']

##Borro las filas que tienen null en device_model y device_language. Juntas son el 5% de las filas en null
installs = installs[pd.notnull(installs['device_model'])]
installs = installs[pd.notnull(installs['device_language'])]

In [4]:
installs['created'] = pd.to_datetime(installs['created'], errors = 'coerce')
installs = installs.sort_values(by = ['ref_hash', 'created'])

In [5]:
events = pd.read_csv('../datos/events.csv',
                     dtype={
                         'device_countrycode': 'category', 
                         'device_city': 'category',
                         'ref_type': 'int64',
                         'application_id': 'int64',
                         'device_os_version': 'category',
                         'device_brand': 'category',
                         'device_model': 'float64',
                         'session_user_agent': 'category',
                         'trans_id': 'category',
                         'user_agent': 'category',
                         'carrier': 'category',
                         'device_os': 'category',
                         'device_os_version': 'category',
                         'device_language': 'category',
                         'connection_type': 'category',
                         'wifi': 'bool'
                     })

In [6]:
del events['attributed']
del events['device_countrycode'] #Borrada porque hay un solo país
del events['trans_id'] #Borrada por tener muy pocos valores (37000/7000000)
del events['event_uuid'] #Borrada por tener todos los valores distintos
del events['device_os_version'] #Borrado por tener 70% de los valores nulos.
del events['device_brand'] #Borrado por tener 67% de los valores nulos
del events['device_city'] #Borrado por tener 76% de los valores nulos
del events['user_agent'] #Borrado por tener 57% de los valores nulos
del events['carrier'] #Borrado por tener 75% de los valores nulos
del events['device_os'] #Borrado por tener 76% de los valores nulos
del events['connection_type'] #Borrado por tener 77% de los valores nulos
del events['session_user_agent']
#------------------------------------------------------------------
events = events[pd.notnull(events['kind'])] #Borrados las filas nulas (0,5%)
#------------------------------------------------------------------
events = events[pd.notnull(events['device_model'])]

In [7]:
events['ref_hash'].value_counts().count()

348229

In [8]:
installs['ref_hash'].value_counts().count()

373799

In [9]:
installs.head()

,created,application_id,ref_type,ref_hash,attributed,implicit,device_model,ip_address,device_language
46742,2019-04-21 19:17:47.657,77,1891515180541284343,40621409780134,False,False,2.019322e+18,990380024993756535,3.3013777759776993e+18
398921,2019-04-18 21:11:50.326,121,1891515180541284343,41863526108385,False,True,1.658417e+18,7530145772806895848,6.977049253562486e+18
249204,2019-04-18 21:11:51.966,121,1891515180541284343,41863526108385,False,False,1.658417e+18,7530145772806895848,6.977049253562486e+18
112218,2019-04-18 21:17:11.946,65,1891515180541284343,41863526108385,False,False,1.658417e+18,7530145772806895848,6.977049253562486e+18
112200,2019-04-18 21:17:16.531,65,1891515180541284343,41863526108385,False,True,1.658417e+18,7530145772806895848,6.977049253562486e+18


In [10]:
eventos = events.groupby('ref_hash').agg({'wifi' : 'mean', 'application_id' : 'count', 'kind' : 'nunique', 'event_id' : 'count'}).reset_index()

In [11]:
events['mode_model'] = events.groupby('ref_hash')['device_model'].agg(lambda x: x.value_counts().index[0])
events['mode_lang'] = events.groupby('ref_hash')['device_language'].agg(lambda x: x.value_counts().index[0])

In [12]:
installs['total_apps'] = installs.groupby('ref_hash')['application_id'].transform('count')
installs['%implicit'] = installs.groupby('ref_hash')['implicit'].transform('mean')
installs['%attributed'] = installs.groupby('ref_hash')['attributed'].transform('mean')
installs['most_freq_lang'] = installs.groupby('ref_hash')['device_language'].transform(lambda x: x.mode().iloc[0])
installs['most_freq_app'] = installs.groupby('ref_hash')['application_id'].transform(lambda x: x.mode().iloc[0])
installs['model'] = installs.groupby('ref_hash')['device_model'].transform(lambda x: x.mode().iloc[0])
installs['ip'] = installs.groupby('ref_hash')['ip_address'].transform(lambda x: x.mode().iloc[0])

In [13]:
installs['events_%wifi'] = installs['ref_hash'].map(eventos.set_index('ref_hash')['wifi'])
installs['event_apps'] = installs['ref_hash'].map(eventos.set_index('ref_hash')['application_id'])
installs['distinct_events'] = installs['ref_hash'].map(eventos.set_index('ref_hash')['kind'])
installs['total_events'] = installs['ref_hash'].map(eventos.set_index('ref_hash')['event_id'])

In [14]:
del installs['attributed']
del installs['implicit']
del installs['application_id']
del installs['device_model']
del installs['ip_address']
del installs['device_language']

In [15]:
installs.fillna(0, inplace=True)

In [16]:
installs = installs.sort_values(by = ['ref_hash', 'created'])

In [17]:
installs['most_freq_lang'] = installs['most_freq_lang'].astype('float64')

In [18]:
installs1 = installs[(installs['created'].dt.day < 21)]
installs2 = installs[(installs['created'].dt.day > 18) & (installs['created'].dt.day < 22)]
installs3 = installs[(installs['created'].dt.day > 19) & (installs['created'].dt.day < 23)]
installs4 = installs[(installs['created'].dt.day > 20) & (installs['created'].dt.day < 24)]
installs5 = installs[(installs['created'].dt.day > 21) & (installs['created'].dt.day < 25)]
installsCheck1 = installs[(installs['created'].dt.day > 20) & (installs['created'].dt.day < 23)]
installsCheck2 = installs[(installs['created'].dt.day > 21) & (installs['created'].dt.day < 24)]
installsCheck3 = installs[(installs['created'].dt.day > 22) & (installs['created'].dt.day < 25)]
installsCheck4 = installs[(installs['created'].dt.day > 23) & (installs['created'].dt.day < 26)]
installsCheck5 = installs[(installs['created'].dt.day > 24) & (installs['created'].dt.day < 27)]

In [19]:
installs1['sc'] = (installs1['created'] - pd.to_datetime('2019-04-18'))/np.timedelta64(1,'s')
installs2['sc'] = (installs2['created'] - pd.to_datetime('2019-04-19'))/np.timedelta64(1,'s')
installs3['sc'] = (installs3['created'] - pd.to_datetime('2019-04-20'))/np.timedelta64(1,'s')
installs4['sc'] = (installs4['created'] - pd.to_datetime('2019-04-21'))/np.timedelta64(1,'s')
installs5['sc'] = (installs5['created'] - pd.to_datetime('2019-04-22'))/np.timedelta64(1,'s')
installsCheck1['sc'] = (installsCheck1['created'] - pd.to_datetime('2019-04-21'))/np.timedelta64(1,'s')
installsCheck2['sc'] = (installsCheck2['created'] - pd.to_datetime('2019-04-22'))/np.timedelta64(1,'s')
installsCheck3['sc'] = (installsCheck3['created'] - pd.to_datetime('2019-04-23'))/np.timedelta64(1,'s')
installsCheck4['sc'] = (installsCheck4['created'] - pd.to_datetime('2019-04-24'))/np.timedelta64(1,'s')
installsCheck5['sc'] = (installsCheck5['created'] - pd.to_datetime('2019-04-25'))/np.timedelta64(1,'s')

/home/gabriel/workspace/7506-1c2019/env/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/gabriel/workspace/7506-1c2019/env/lib/python3.5/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/gabriel/workspace/7506-1c2019/env/lib/python3.5/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_ind

In [20]:
installs1 = installs1.drop_duplicates('ref_hash', keep = 'first')
installs2 = installs2.drop_duplicates('ref_hash', keep = 'first')
installs3 = installs3.drop_duplicates('ref_hash', keep = 'first')
installs4 = installs4.drop_duplicates('ref_hash', keep = 'first')
installs5 = installs5.drop_duplicates('ref_hash', keep = 'first')
installsCheck1 = installsCheck1.drop_duplicates('ref_hash', keep = 'first')
installsCheck2 = installsCheck2.drop_duplicates('ref_hash', keep = 'first')
installsCheck3 = installsCheck3.drop_duplicates('ref_hash', keep = 'first')
installsCheck4 = installsCheck4.drop_duplicates('ref_hash', keep = 'first')
installsCheck5 = installsCheck5.drop_duplicates('ref_hash', keep = 'first')

In [21]:
del installs1['created']
del installs2['created']
del installs3['created']
del installs4['created']
del installs5['created']
del installsCheck1['created']
del installsCheck2['created']
del installsCheck3['created']
del installsCheck4['created']
del installsCheck5['created']

# Algoritmos

### XGBoost

In [22]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error

#### Ventana 1

In [23]:
X, y = installs1.iloc[:,:-1], installs1.iloc[:,-1]
data_dmatrix = xgb.DMatrix(data=X,label=y)
xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', 
                colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha =10, n_estimators = 11)
xg_reg.fit(X, y)
preds = xg_reg.predict(installsCheck1.iloc[:,:-1])
rmse = np.sqrt(mean_squared_error(installsCheck1.iloc[:,-1], preds))
print("RMSE: %f" % (rmse))

/home/gabriel/workspace/7506-1c2019/env/lib/python3.5/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/home/gabriel/workspace/7506-1c2019/env/lib/python3.5/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


RMSE: 52221.659961


#### Ventana 2

In [24]:
X, y = installs2.iloc[:,:-1], installs2.iloc[:,-1]
data_dmatrix = xgb.DMatrix(data=X,label=y)
xg_reg.fit(X, y)
preds = xg_reg.predict(installsCheck2.iloc[:,:-1])
rmse = np.sqrt(mean_squared_error(installsCheck2.iloc[:,-1], preds))
print("RMSE: %f" % (rmse))

/home/gabriel/workspace/7506-1c2019/env/lib/python3.5/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/home/gabriel/workspace/7506-1c2019/env/lib/python3.5/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


RMSE: 52146.316026


#### Ventana 3

In [25]:
X, y = installs3.iloc[:,:-1], installs3.iloc[:,-1]
data_dmatrix = xgb.DMatrix(data=X,label=y)
xg_reg.fit(X, y)
preds = xg_reg.predict(installsCheck3.iloc[:,:-1])
rmse = np.sqrt(mean_squared_error(installsCheck3.iloc[:,-1], preds))
print("RMSE: %f" % (rmse))

/home/gabriel/workspace/7506-1c2019/env/lib/python3.5/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/home/gabriel/workspace/7506-1c2019/env/lib/python3.5/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


RMSE: 52157.097772


#### Ventana 4

In [26]:
X, y = installs4.iloc[:,:-1], installs4.iloc[:,-1]
data_dmatrix = xgb.DMatrix(data=X,label=y)
xg_reg.fit(X, y)
preds = xg_reg.predict(installsCheck4.iloc[:,:-1])
rmse = np.sqrt(mean_squared_error(installsCheck4.iloc[:,-1], preds))
print("RMSE: %f" % (rmse))

/home/gabriel/workspace/7506-1c2019/env/lib/python3.5/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/home/gabriel/workspace/7506-1c2019/env/lib/python3.5/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


RMSE: 53224.249049


#### Ventana 5

In [27]:
X, y = installs5.iloc[:,:-1], installs5.iloc[:,-1]
data_dmatrix = xgb.DMatrix(data=X,label=y)
xg_reg.fit(X, y)
preds = xg_reg.predict(installsCheck5.iloc[:,:-1])
rmse = np.sqrt(mean_squared_error(installsCheck5.iloc[:,-1], preds))
print("RMSE: %f" % (rmse))

/home/gabriel/workspace/7506-1c2019/env/lib/python3.5/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/home/gabriel/workspace/7506-1c2019/env/lib/python3.5/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


RMSE: 51092.387332


In [53]:
from joblib import dump 
import os
directory = 'modelos/sc'
if not os.path.exists(directory):
    os.makedirs(directory)
# Guardo modelo como un pickle file 
dump(xg_reg, 'modelos/sc/xg_reg.pkl')

['modelos/sc/xg_reg.pkl']

#### Target

In [28]:
target = pd.read_csv('../datos/target_competencia_ids.csv')

In [29]:
targets_sc = target[target['ref_hash'].str.contains('_sc')]
targets_st = target[target['ref_hash'].str.contains('_st')]

In [30]:
targets_sc['ref_hash'] = targets_sc['ref_hash'].map(lambda x: str(x)[:-3])
targets_sc['ref_hash'] = targets_sc['ref_hash'].astype(np.int64)

/home/gabriel/workspace/7506-1c2019/env/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/gabriel/workspace/7506-1c2019/env/lib/python3.5/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [31]:
installs_sin_dup = installs.drop_duplicates('ref_hash')

In [32]:
targets_sc_with_installs = targets_sc.merge(installs_sin_dup, how = 'left', on = 'ref_hash')

In [33]:
events.drop_duplicates('ref_hash', inplace= True)

In [34]:
nulls = targets_sc_with_installs[targets_sc_with_installs['created'].isnull()]
nulls['ref_type'] = nulls['ref_hash'].map(events.set_index('ref_hash')['ref_type'])
nulls['model'] = nulls['ref_hash'].map(events.set_index('ref_hash')['device_model'])
nulls['most_freq_lang'] = nulls['ref_hash'].map(events.set_index('ref_hash')['device_language'])
nulls['most_freq_app'] = nulls['ref_hash'].map(events.set_index('ref_hash')['application_id'])
nulls['ip'] = nulls['ref_hash'].map(events.set_index('ref_hash')['ip_address'])
nulls['events_%wifi'] = nulls['ref_hash'].map(eventos.set_index('ref_hash')['wifi'])
nulls['event_apps'] = nulls['ref_hash'].map(eventos.set_index('ref_hash')['application_id'])
nulls['distinct_events'] = nulls['ref_hash'].map(eventos.set_index('ref_hash')['kind'])
nulls['total_events'] = nulls['ref_hash'].map(eventos.set_index('ref_hash')['event_id'])

/home/gabriel/workspace/7506-1c2019/env/lib/python3.5/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/gabriel/workspace/7506-1c2019/env/lib/python3.5/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/gabriel/workspace/7506-1c2019/env/lib/python3.5/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Tr

In [35]:
targets_sc_with_installs.update(nulls)
targets_sc_with_installs.head()

,ref_hash,obj,created,ref_type,total_apps,%implicit,%attributed,most_freq_lang,most_freq_app,model,ip,events_%wifi,event_apps,distinct_events,total_events
0,1.000169e+18,0.0,2019-04-21 08:14:06.178,1.891515e+18,1.0,0.0,0.0,3.30138e+18,122.0,1.805456e+18,5.483720e+18,0.000000,32.0,4.0,32.0
1,1.000396e+18,0.0,2019-04-20 05:59:26.866,1.891515e+18,2.0,0.5,0.0,6.97705e+18,121.0,8.058599e+17,6.126933e+18,1.000000,2.0,2.0,2.0
2,1.003027e+18,0.0,2019-04-23 20:37:24.299,1.891515e+18,2.0,0.5,0.0,3.30138e+18,14.0,8.590181e+18,2.036110e+18,0.833333,54.0,10.0,54.0
3,1.006670e+18,0.0,2019-04-21 01:20:43.968,1.494519e+18,2.0,0.5,0.0,8.44142e+18,68.0,6.794880e+18,2.799308e+18,0.333333,3.0,2.0,3.0
4,1.007573e+18,0.0,2019-04-18 07:32:44.992,1.891515e+18,1.0,0.0,0.0,3.30138e+18,94.0,1.327864e+18,8.238829e+18,0.000000,6.0,1.0,6.0


In [36]:
targets_sc_with_installs.fillna({'total_apps' : 0, 'ref_type' : 0, '%attributed' : 0, '%implicit': 0, 'model' : 0,
                                 'most_freq_lang' : 0, 'ip' : 0, 'events_%wifi' : 0, 'event_apps' : 0, 
                                 'distinct_events' : 0, 'total_events' : 0, 'most_freq_app' : 0}, inplace = True)

In [37]:
del targets_sc_with_installs['created']
del targets_sc_with_installs['obj']

In [38]:
targets_sc_with_installs['most_freq_lang'] = targets_sc_with_installs['most_freq_lang'].astype('float64')
X = targets_sc_with_installs[['ref_type', 'ref_hash', 'total_apps', '%implicit', '%attributed', 'most_freq_lang', 
                             'most_freq_app', 'model', 'ip', 'events_%wifi', 'event_apps', 'distinct_events', 
                             'total_events']]
predictions_sc_xgb = xg_reg.predict(X)

In [40]:
predictions_sc_xgb

array([85503.766, 90008.85 , 84732.516, ..., 87279.84 , 89796.516,
       86535.55 ], dtype=float32)

In [41]:
target

,ref_hash,obj
0,1000169251625791246_sc,0
1,1000169251625791246_st,0
2,1000395625957344683_sc,0
3,1000395625957344683_st,0
4,1003027494996471685_sc,0
5,1003027494996471685_st,0
6,1006670001679961544_sc,0
7,1006670001679961544_st,0
8,1007573308966476713_sc,0
9,1007573308966476713_st,0


In [42]:
# sc_pred = pd.read_csv('../datos/PrediccionesKNN-RF-XGBoost.txt', header = None)
# sc_pred.index = np.arange(1, 8074, 2)
target['obj'] = predictions_sc_xgb[0]
target.update(targets_st)
target

,ref_hash,obj
0,1000169251625791246_sc,85503.765625
1,1000169251625791246_st,0.000000
2,1000395625957344683_sc,85503.765625
3,1000395625957344683_st,0.000000
4,1003027494996471685_sc,85503.765625
5,1003027494996471685_st,0.000000
6,1006670001679961544_sc,85503.765625
7,1006670001679961544_st,0.000000
8,1007573308966476713_sc,85503.765625
9,1007573308966476713_st,0.000000


In [43]:
target.to_csv('../datos/preds-xgboost-rf-knn-nuevos-features.csv', index = False)

### Bagging

In [44]:
from sklearn.ensemble import BaggingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor

In [45]:
X, y = installs1.iloc[:,:-1], installs1.iloc[:,-1]
bagging = BaggingRegressor(KNeighborsRegressor(), max_samples=0.5, max_features=0.5)
bagging.fit(X, y)
preds = bagging.predict(installsCheck1.iloc[:,:-1])
rmse = np.sqrt(mean_squared_error(installsCheck1.iloc[:,-1], preds))
print("RMSE: %f" % (rmse))

RMSE: 67793.738466


In [46]:
X, y = installs2.iloc[:,:-1], installs2.iloc[:,-1]
bagging.fit(X, y)
preds = bagging.predict(installsCheck2.iloc[:,:-1])
rmse = np.sqrt(mean_squared_error(installsCheck2.iloc[:,-1], preds))
print("RMSE: %f" % (rmse))

RMSE: 69484.395435


In [47]:
X, y = installs3.iloc[:,:-1], installs3.iloc[:,-1]
bagging.fit(X, y)
preds = bagging.predict(installsCheck3.iloc[:,:-1])
rmse = np.sqrt(mean_squared_error(installsCheck3.iloc[:,-1], preds))
print("RMSE: %f" % (rmse))

RMSE: 70109.334570


In [48]:
X, y = installs4.iloc[:,:-1], installs4.iloc[:,-1]
bagging.fit(X, y)
preds = bagging.predict(installsCheck4.iloc[:,:-1])
rmse = np.sqrt(mean_squared_error(installsCheck4.iloc[:,-1], preds))
print("RMSE: %f" % (rmse))

RMSE: 69036.720016


In [49]:
X, y = installs5.iloc[:,:-1], installs5.iloc[:,-1]
bagging.fit(X, y)
preds = bagging.predict(installsCheck5.iloc[:,:-1])
rmse = np.sqrt(mean_squared_error(installsCheck5.iloc[:,-1], preds))
print("RMSE: %f" % (rmse))

RMSE: 66388.241608


In [50]:
X = targets_sc_with_installs[['ref_type', 'ref_hash', 'total_apps', '%implicit', '%attributed', 'most_freq_lang', 
                             'most_freq_app', 'model', 'ip', 'events_%wifi', 'event_apps', 'distinct_events', 
                             'total_events']]
predictions_sc_bagging = bagging.predict(X)

In [54]:
dump(bagging, 'modelos/sc/bagging.pkl')

['modelos/sc/bagging.pkl']

### AdaBoost

In [55]:
X, y = installs1.iloc[:,:-1], installs1.iloc[:,-1]
regr = AdaBoostRegressor(random_state=0, n_estimators=50, learning_rate=0.1)
regr.fit(X, y) 
preds = regr.predict(installsCheck1.iloc[:,:-1])
rmse = np.sqrt(mean_squared_error(installsCheck1.iloc[:,-1], preds))
print("RMSE: %f" % (rmse))

RMSE: 66205.001008


In [56]:
X, y = installs2.iloc[:,:-1], installs2.iloc[:,-1]
regr.fit(X, y) 
preds = regr.predict(installsCheck2.iloc[:,:-1])
rmse = np.sqrt(mean_squared_error(installsCheck2.iloc[:,-1], preds))
print("RMSE: %f" % (rmse))

RMSE: 67957.789315


In [57]:
X, y = installs3.iloc[:,:-1], installs3.iloc[:,-1]
regr.fit(X, y) 
preds = regr.predict(installsCheck3.iloc[:,:-1])
rmse = np.sqrt(mean_squared_error(installsCheck3.iloc[:,-1], preds))
print("RMSE: %f" % (rmse))

RMSE: 68742.307108


In [58]:
X, y = installs4.iloc[:,:-1], installs4.iloc[:,-1]
regr.fit(X, y) 
preds = regr.predict(installsCheck4.iloc[:,:-1])
rmse = np.sqrt(mean_squared_error(installsCheck4.iloc[:,-1], preds))
print("RMSE: %f" % (rmse))

RMSE: 67266.935307


In [59]:
X, y = installs5.iloc[:,:-1], installs5.iloc[:,-1]
regr.fit(X, y) 
preds = regr.predict(installsCheck5.iloc[:,:-1])
rmse = np.sqrt(mean_squared_error(installsCheck5.iloc[:,-1], preds))
print("RMSE: %f" % (rmse))

RMSE: 64948.432210


In [60]:
targets_sc_with_installs['most_freq_lang'] = targets_sc_with_installs['most_freq_lang'].astype('float64')
X = targets_sc_with_installs[['ref_type', 'ref_hash', 'total_apps', '%implicit', '%attributed', 'most_freq_lang', 
                             'most_freq_app', 'model', 'ip', 'events_%wifi', 'event_apps', 'distinct_events', 
                             'total_events']]
predictions_sc_adaboost = regr.predict(X)

In [61]:
dump(regr, 'modelos/sc/regr.pkl')

['modelos/sc/regr.pkl']

### LGBM

In [62]:
import lightgbm as lgb

In [63]:
lgb_train = lgb.Dataset(installs1.iloc[:,:-1], installs1.iloc[:,-1])
lgb_eval = lgb.Dataset(installsCheck1.iloc[:,:-1], installsCheck1.iloc[:,-1], reference=lgb_train)
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2', 'l1'},
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}
gbm = lgb.train(params, lgb_train, num_boost_round=20, valid_sets=lgb_eval, early_stopping_rounds=10)
preds = gbm.predict(installsCheck1.iloc[:,:-1], num_iteration=gbm.best_iteration)
rmse = np.sqrt(mean_squared_error(installsCheck1.iloc[:,-1], preds))
print("RMSE: %f" % (rmse))

[1]	valid_0's l1: 55436.3	valid_0's l2: 4.32643e+09
Training until validation scores don't improve for 10 rounds.
[2]	valid_0's l1: 55438.4	valid_0's l2: 4.32734e+09
[3]	valid_0's l1: 55433.8	valid_0's l2: 4.32699e+09
[4]	valid_0's l1: 55435.3	valid_0's l2: 4.32795e+09
[5]	valid_0's l1: 55437	valid_0's l2: 4.3291e+09
[6]	valid_0's l1: 55446.4	valid_0's l2: 4.33147e+09
[7]	valid_0's l1: 55453.3	valid_0's l2: 4.33372e+09
[8]	valid_0's l1: 55462.3	valid_0's l2: 4.33608e+09
[9]	valid_0's l1: 55473	valid_0's l2: 4.33856e+09
[10]	valid_0's l1: 55483.8	valid_0's l2: 4.34101e+09
[11]	valid_0's l1: 55492.2	valid_0's l2: 4.3434e+09
Early stopping, best iteration is:
[1]	valid_0's l1: 55436.3	valid_0's l2: 4.32643e+09
RMSE: 65775.576436


In [64]:
lgb_train = lgb.Dataset(installs2.iloc[:,:-1], installs2.iloc[:,-1])
lgb_eval = lgb.Dataset(installsCheck2.iloc[:,:-1], installsCheck2.iloc[:,-1], reference=lgb_train)
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2', 'l1'},
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}
gbm = lgb.train(params, lgb_train, num_boost_round=20, valid_sets=lgb_eval, early_stopping_rounds=10)
preds = gbm.predict(installsCheck2.iloc[:,:-1], num_iteration=gbm.best_iteration)
rmse = np.sqrt(mean_squared_error(installsCheck2.iloc[:,-1], preds))
print("RMSE: %f" % (rmse))

[1]	valid_0's l1: 56837.9	valid_0's l2: 4.60391e+09
Training until validation scores don't improve for 10 rounds.
[2]	valid_0's l1: 56843.4	valid_0's l2: 4.60514e+09
[3]	valid_0's l1: 56841.2	valid_0's l2: 4.60507e+09
[4]	valid_0's l1: 56835.3	valid_0's l2: 4.60493e+09
[5]	valid_0's l1: 56838.4	valid_0's l2: 4.60613e+09
[6]	valid_0's l1: 56841.7	valid_0's l2: 4.60709e+09
[7]	valid_0's l1: 56847.4	valid_0's l2: 4.60841e+09
[8]	valid_0's l1: 56846.6	valid_0's l2: 4.6088e+09
[9]	valid_0's l1: 56852.1	valid_0's l2: 4.61026e+09
[10]	valid_0's l1: 56860.1	valid_0's l2: 4.61221e+09
[11]	valid_0's l1: 56855.8	valid_0's l2: 4.61175e+09
Early stopping, best iteration is:
[1]	valid_0's l1: 56837.9	valid_0's l2: 4.60391e+09
RMSE: 67852.095327


In [65]:
lgb_train = lgb.Dataset(installs3.iloc[:,:-1], installs3.iloc[:,-1])
lgb_eval = lgb.Dataset(installsCheck3.iloc[:,:-1], installsCheck3.iloc[:,-1], reference=lgb_train)
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2', 'l1'},
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}
gbm = lgb.train(params, lgb_train, num_boost_round=20, valid_sets=lgb_eval, early_stopping_rounds=10)
preds = gbm.predict(installsCheck3.iloc[:,:-1], num_iteration=gbm.best_iteration)
rmse = np.sqrt(mean_squared_error(installsCheck3.iloc[:,-1], preds))
print("RMSE: %f" % (rmse))

[1]	valid_0's l1: 57631.5	valid_0's l2: 4.7152e+09
Training until validation scores don't improve for 10 rounds.
[2]	valid_0's l1: 57639.1	valid_0's l2: 4.71753e+09
[3]	valid_0's l1: 57639.4	valid_0's l2: 4.71796e+09
[4]	valid_0's l1: 57639.1	valid_0's l2: 4.71891e+09
[5]	valid_0's l1: 57647.1	valid_0's l2: 4.72151e+09
[6]	valid_0's l1: 57656.3	valid_0's l2: 4.72429e+09
[7]	valid_0's l1: 57665.1	valid_0's l2: 4.72704e+09
[8]	valid_0's l1: 57667.8	valid_0's l2: 4.72857e+09
[9]	valid_0's l1: 57680.3	valid_0's l2: 4.73201e+09
[10]	valid_0's l1: 57691.3	valid_0's l2: 4.7353e+09
[11]	valid_0's l1: 57698.8	valid_0's l2: 4.73752e+09
Early stopping, best iteration is:
[1]	valid_0's l1: 57631.5	valid_0's l2: 4.7152e+09
RMSE: 68667.326555


In [66]:
lgb_train = lgb.Dataset(installs4.iloc[:,:-1], installs4.iloc[:,-1])
lgb_eval = lgb.Dataset(installsCheck4.iloc[:,:-1], installsCheck4.iloc[:,-1], reference=lgb_train)
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2', 'l1'},
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}
gbm = lgb.train(params, lgb_train, num_boost_round=20, valid_sets=lgb_eval, early_stopping_rounds=10)
preds = gbm.predict(installsCheck4.iloc[:,:-1], num_iteration=gbm.best_iteration)
rmse = np.sqrt(mean_squared_error(installsCheck4.iloc[:,-1], preds))
print("RMSE: %f" % (rmse))

[1]	valid_0's l1: 56382.1	valid_0's l2: 4.53583e+09
Training until validation scores don't improve for 10 rounds.
[2]	valid_0's l1: 56390.9	valid_0's l2: 4.53795e+09
[3]	valid_0's l1: 56393.7	valid_0's l2: 4.53819e+09
[4]	valid_0's l1: 56402.9	valid_0's l2: 4.53998e+09
[5]	valid_0's l1: 56410.6	valid_0's l2: 4.54169e+09
[6]	valid_0's l1: 56427.7	valid_0's l2: 4.54482e+09
[7]	valid_0's l1: 56443.3	valid_0's l2: 4.54776e+09
[8]	valid_0's l1: 56455.5	valid_0's l2: 4.55008e+09
[9]	valid_0's l1: 56462.7	valid_0's l2: 4.55204e+09
[10]	valid_0's l1: 56472.5	valid_0's l2: 4.55389e+09
[11]	valid_0's l1: 56487.6	valid_0's l2: 4.55734e+09
Early stopping, best iteration is:
[1]	valid_0's l1: 56382.1	valid_0's l2: 4.53583e+09
RMSE: 67348.581056


In [67]:
lgb_train = lgb.Dataset(installs5.iloc[:,:-1], installs5.iloc[:,-1])
lgb_eval = lgb.Dataset(installsCheck5.iloc[:,:-1], installsCheck5.iloc[:,-1], reference=lgb_train)
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2', 'l1'},
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}
gbm = lgb.train(params, lgb_train, num_boost_round=20, valid_sets=lgb_eval, early_stopping_rounds=10)
preds = gbm.predict(installsCheck5.iloc[:,:-1], num_iteration=gbm.best_iteration)
rmse = np.sqrt(mean_squared_error(installsCheck5.iloc[:,-1], preds))
print("RMSE: %f" % (rmse))

[1]	valid_0's l1: 54171.5	valid_0's l2: 4.16867e+09
Training until validation scores don't improve for 10 rounds.
[2]	valid_0's l1: 54199.4	valid_0's l2: 4.17498e+09
[3]	valid_0's l1: 54193.9	valid_0's l2: 4.17481e+09
[4]	valid_0's l1: 54221.2	valid_0's l2: 4.18067e+09
[5]	valid_0's l1: 54219.4	valid_0's l2: 4.18187e+09
[6]	valid_0's l1: 54221.2	valid_0's l2: 4.18366e+09
[7]	valid_0's l1: 54221.1	valid_0's l2: 4.18519e+09
[8]	valid_0's l1: 54221.9	valid_0's l2: 4.18683e+09
[9]	valid_0's l1: 54219	valid_0's l2: 4.1879e+09
[10]	valid_0's l1: 54237.4	valid_0's l2: 4.19224e+09
[11]	valid_0's l1: 54259.8	valid_0's l2: 4.1977e+09
Early stopping, best iteration is:
[1]	valid_0's l1: 54171.5	valid_0's l2: 4.16867e+09
RMSE: 64565.240891


In [68]:
targets_sc_with_installs['most_freq_lang'] = targets_sc_with_installs['most_freq_lang'].astype('float64')
X = targets_sc_with_installs[['ref_type', 'ref_hash', 'total_apps', '%implicit', '%attributed', 'most_freq_lang', 
                             'most_freq_app', 'model', 'ip', 'events_%wifi', 'event_apps', 'distinct_events', 
                             'total_events']]
predictions_sc_lgbm = gbm.predict(X)

In [69]:
dump(gbm, 'modelos/sc/gbm.pkl')

['modelos/sc/gbm.pkl']

### Random Forests

In [70]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators=10, criterion='mse', max_depth=5, min_samples_split=2, 
                           min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features='auto', 
                           max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, 
                           bootstrap=True, oob_score=False, n_jobs=None, random_state=None, verbose=0, 
                           warm_start=False)

In [71]:
X, y = installs1.iloc[:,:-1], installs1.iloc[:,-1]
rf.fit(X, y)
preds = rf.predict(installsCheck1.iloc[:,:-1])
rmse = np.sqrt(mean_squared_error(installsCheck1.iloc[:,-1], preds))
print("RMSE: %f" % (rmse))

RMSE: 66241.227820


In [72]:
X, y = installs2.iloc[:,:-1], installs2.iloc[:,-1]
rf.fit(X, y)
preds = rf.predict(installsCheck2.iloc[:,:-1])
rmse = np.sqrt(mean_squared_error(installsCheck2.iloc[:,-1], preds))
print("RMSE: %f" % (rmse))

RMSE: 67847.388428


In [73]:
X, y = installs3.iloc[:,:-1], installs3.iloc[:,-1]
rf.fit(X, y)
preds = rf.predict(installsCheck3.iloc[:,:-1])
rmse = np.sqrt(mean_squared_error(installsCheck3.iloc[:,-1], preds))
print("RMSE: %f" % (rmse))

RMSE: 68859.282962


In [74]:
X, y = installs4.iloc[:,:-1], installs4.iloc[:,-1]
rf.fit(X, y)
preds = rf.predict(installsCheck4.iloc[:,:-1])
rmse = np.sqrt(mean_squared_error(installsCheck4.iloc[:,-1], preds))
print("RMSE: %f" % (rmse))

RMSE: 67523.298430


In [75]:
X, y = installs5.iloc[:,:-1], installs5.iloc[:,-1]
rf.fit(X, y)
preds = rf.predict(installsCheck5.iloc[:,:-1])
rmse = np.sqrt(mean_squared_error(installsCheck5.iloc[:,-1], preds))
print("RMSE: %f" % (rmse))

RMSE: 64671.490337


In [76]:
targets_sc_with_installs['most_freq_lang'] = targets_sc_with_installs['most_freq_lang'].astype('float64')
X = targets_sc_with_installs[['ref_type', 'ref_hash', 'total_apps', '%implicit', '%attributed', 'most_freq_lang', 
                             'most_freq_app', 'model', 'ip', 'events_%wifi', 'event_apps', 'distinct_events', 
                             'total_events']]
predictions_sc_rf = rf.predict(X)

In [77]:
dump(rf, 'modelos/sc/rf.pkl')

['modelos/sc/rf.pkl']

### MLP

In [78]:
from sklearn.neural_network import MLPRegressor
nn = MLPRegressor(activation='logistic', learning_rate='adaptive', warm_start=True, max_iter= 400)

In [79]:
X, y = installs1.iloc[:,:-1], installs1.iloc[:,-1]
nn.fit(X, y)
preds = nn.predict(installsCheck1.iloc[:,:-1])
rmse = np.sqrt(mean_squared_error(installsCheck1.iloc[:,-1], preds))
print("RMSE: %f" % (rmse))

/home/gabriel/workspace/7506-1c2019/env/lib/python3.5/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (400) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


RMSE: 84109.713272


In [80]:
X, y = installs2.iloc[:,:-1], installs2.iloc[:,-1]
nn.fit(X, y)
preds = nn.predict(installsCheck2.iloc[:,:-1])
rmse = np.sqrt(mean_squared_error(installsCheck2.iloc[:,-1], preds))
print("RMSE: %f" % (rmse))

RMSE: 82863.262654


In [81]:
X, y = installs3.iloc[:,:-1], installs3.iloc[:,-1]
nn.fit(X, y)
preds = nn.predict(installsCheck3.iloc[:,:-1])
rmse = np.sqrt(mean_squared_error(installsCheck3.iloc[:,-1], preds))
print("RMSE: %f" % (rmse))

RMSE: 81846.114578


In [82]:
X, y = installs4.iloc[:,:-1], installs4.iloc[:,-1]
nn.fit(X, y)
preds = nn.predict(installsCheck4.iloc[:,:-1])
rmse = np.sqrt(mean_squared_error(installsCheck4.iloc[:,-1], preds))
print("RMSE: %f" % (rmse))

RMSE: 84326.465828


In [83]:
X, y = installs5.iloc[:,:-1], installs5.iloc[:,-1]
nn.fit(X, y)
preds = nn.predict(installsCheck5.iloc[:,:-1])
rmse = np.sqrt(mean_squared_error(installsCheck5.iloc[:,-1], preds))
print("RMSE: %f" % (rmse))

RMSE: 79019.263483


In [84]:
targets_sc_with_installs['most_freq_lang'] = targets_sc_with_installs['most_freq_lang'].astype('float64')
X = targets_sc_with_installs[['ref_type', 'ref_hash', 'total_apps', '%implicit', '%attributed', 'most_freq_lang', 
                             'most_freq_app', 'model', 'ip', 'events_%wifi', 'event_apps', 'distinct_events', 
                             'total_events']]
predictions_sc_nn = nn.predict(X)

In [85]:
dump(nn, 'modelos/sc/nn.pkl')

['modelos/sc/nn.pkl']

## Promedio las predicciones y actualizo el target

In [87]:
predictions_sc = np.add(np.add(np.add(np.add(np.add(predictions_sc_xgb,predictions_sc_bagging), predictions_sc_adaboost), predictions_sc_lgbm),predictions_sc_rf), predictions_sc_nn)
predictions_sc = predictions_sc / 6
targets_sc['obj'] = predictions_sc
predicciones_target_sc = target[target['ref_hash'].str.contains('_sc')]
predicciones_target_sc['obj'] = predictions_sc
target.update(predicciones_target_sc)
# sc_pred = pd.read_csv('../datos/PrediccionesKNN-RF-XGBoost.txt', header = None)
# sc_pred.index = np.arange(1, 8074, 2)
# targets_st['obj'] = sc_pred[0]
# target.update(targets_st)
target

/home/gabriel/workspace/7506-1c2019/env/lib/python3.5/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/gabriel/workspace/7506-1c2019/env/lib/python3.5/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


,ref_hash,obj
0,1000169251625791246_sc,105747.054158
1,1000169251625791246_st,0.000000
2,1000395625957344683_sc,110877.400863
3,1000395625957344683_st,0.000000
4,1003027494996471685_sc,107322.915289
5,1003027494996471685_st,0.000000
6,1006670001679961544_sc,108395.398453
7,1006670001679961544_st,0.000000
8,1007573308966476713_sc,107132.568269
9,1007573308966476713_st,0.000000


In [88]:
target.to_csv('../datos/preds-promedios.csv', index = False)

In [89]:
target.count()

ref_hash    8074
obj         8074
dtype: int64

## Blending

In [90]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [91]:
xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', 
                colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha =10, n_estimators = 11)
regr = AdaBoostRegressor(random_state=0, n_estimators=50, learning_rate=0.1)
rf = RandomForestRegressor(n_estimators=10, criterion='mse', max_depth=5, min_samples_split=2, 
                           min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features='auto', 
                           max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, 
                           bootstrap=True, oob_score=False, n_jobs=None, random_state=None, verbose=0, 
                           warm_start=False)
bagging = BaggingRegressor(KNeighborsRegressor(), max_samples=0.5, max_features=0.5)
gbm = lgb.LGBMRegressor(boosting_type= 'gbdt',
                        objective= 'regression',
                        num_leaves= 31,
                        learning_rate= 0.05,
                        feature_fraction= 0.9,
                        bagging_fraction= 0.8,
                        bagging_freq= 5,
                        verbose= 0)

#### Ventana 1

In [92]:
X_test, y_test = installsCheck1.iloc[:,:-1], installsCheck1.iloc[:,-1]

In [93]:
X, y = installs1.iloc[:,:-1], installs1.iloc[:,-1]
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3)
xg_reg.fit(X_train, y_train)
rf.fit(X_train, y_train)
bagging.fit(X_train, y_train)
gbm.fit(X_train, y_train, eval_set=[(X_val, y_val)], eval_metric='l1', early_stopping_rounds=15)

/home/gabriel/workspace/7506-1c2019/env/lib/python3.5/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[1]	valid_0's l2: 5.90274e+09	valid_0's l1: 67216
Training until validation scores don't improve for 15 rounds.
[2]	valid_0's l2: 5.89458e+09	valid_0's l1: 67170.8
[3]	valid_0's l2: 5.89294e+09	valid_0's l1: 67161.2
[4]	valid_0's l2: 5.88585e+09	valid_0's l1: 67120.1
[5]	valid_0's l2: 5.87986e+09	valid_0's l1: 67083
[6]	valid_0's l2: 5.87369e+09	valid_0's l1: 67044.2
[7]	valid_0's l2: 5.86855e+09	valid_0's l1: 67010.3
[8]	valid_0's l2: 5.86401e+09	valid_0's l1: 66979
[9]	valid_0's l2: 5.85982e+09	valid_0's l1: 66949.2
[10]	valid_0's l2: 5.85652e+09	valid_0's l1: 66925.1
[11]	valid_0's l2: 5.85326e+09	valid_0's l1: 66902.4
[12]	valid_0's l2: 5.85029e+09	valid_0's l1: 66880.6
[13]	valid_0's l2: 5.84756e+09	valid_0's l1: 66860.3
[14]	valid_0's l2: 5.84552e+09	valid_0's l1: 66844.5
[15]	valid_0's l2: 5.84338e+09	valid_0's l1: 66827.7
[16]	valid_0's l2: 5.84088e+09	valid_0's l1: 66808
[17]	valid_0's l2: 5.83854e+09	valid_0's l1: 66789.9
[18]	valid_0's l2: 5.8371e+09	valid_0's l1: 66778.2
[1

LGBMRegressor(bagging_fraction=0.8, bagging_freq=5, boosting_type='gbdt',
              class_weight=None, colsample_bytree=1.0, feature_fraction=0.9,
              importance_type='split', learning_rate=0.05, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=100, n_jobs=-1, num_leaves=31,
              objective='regression', random_state=None, reg_alpha=0.0,
              reg_lambda=0.0, silent=True, subsample=1.0,
              subsample_for_bin=200000, subsample_freq=0, verbose=0)

In [94]:
preds_xgb = xg_reg.predict(X_val)
preds_rf = rf.predict(X_val)
preds_bag = bagging.predict(X_val)
preds_lgbm = gbm.predict(X_val)
test_preds_xgb = xg_reg.predict(X_test)
test_preds_rf = rf.predict(X_test)
test_preds_bag = bagging.predict(X_test)
test_preds_lgbm = gbm.predict(X_test)

In [95]:
X_val['pred_xgb'] = preds_xgb
X_val['pred_rf'] = preds_rf
X_val['pred_bag'] = preds_bag
X_val['pred_lgbm'] = preds_lgbm
X_test['pred_xgb'] = test_preds_xgb
X_test['pred_rf'] = test_preds_rf
X_test['pred_bag'] = test_preds_bag
X_test['pred_lgbm'] = test_preds_lgbm

/home/gabriel/workspace/7506-1c2019/env/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/gabriel/workspace/7506-1c2019/env/lib/python3.5/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/gabriel/workspace/7506-1c2019/env/lib/python3.5/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_ind

In [96]:
regr.fit(X_val, y_val)
preds = regr.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 66716.667669


#### Ventana 2

In [97]:
X_test, y_test = installsCheck2.iloc[:,:-1], installsCheck2.iloc[:,-1]
X, y = installs2.iloc[:,:-1], installs2.iloc[:,-1]
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3)
xg_reg.fit(X_train, y_train)
rf.fit(X_train, y_train)
bagging.fit(X_train, y_train)
gbm.fit(X_train, y_train, eval_set=[(X_val, y_val)], eval_metric='l1', early_stopping_rounds=15)

/home/gabriel/workspace/7506-1c2019/env/lib/python3.5/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[1]	valid_0's l2: 5.8809e+09	valid_0's l1: 67036.3
Training until validation scores don't improve for 15 rounds.
[2]	valid_0's l2: 5.8791e+09	valid_0's l1: 67026.5
[3]	valid_0's l2: 5.87854e+09	valid_0's l1: 67022.4
[4]	valid_0's l2: 5.877e+09	valid_0's l1: 67012.7
[5]	valid_0's l2: 5.87519e+09	valid_0's l1: 67002.4
[6]	valid_0's l2: 5.87372e+09	valid_0's l1: 66993.4
[7]	valid_0's l2: 5.87196e+09	valid_0's l1: 66982.7
[8]	valid_0's l2: 5.8712e+09	valid_0's l1: 66977.2
[9]	valid_0's l2: 5.87049e+09	valid_0's l1: 66970.7
[10]	valid_0's l2: 5.86959e+09	valid_0's l1: 66964.2
[11]	valid_0's l2: 5.86806e+09	valid_0's l1: 66955.1
[12]	valid_0's l2: 5.86693e+09	valid_0's l1: 66947.6
[13]	valid_0's l2: 5.8666e+09	valid_0's l1: 66944.4
[14]	valid_0's l2: 5.86588e+09	valid_0's l1: 66938.5
[15]	valid_0's l2: 5.86533e+09	valid_0's l1: 66934.2
[16]	valid_0's l2: 5.86504e+09	valid_0's l1: 66931.5
[17]	valid_0's l2: 5.86479e+09	valid_0's l1: 66928.2
[18]	valid_0's l2: 5.86462e+09	valid_0's l1: 66925.8

LGBMRegressor(bagging_fraction=0.8, bagging_freq=5, boosting_type='gbdt',
              class_weight=None, colsample_bytree=1.0, feature_fraction=0.9,
              importance_type='split', learning_rate=0.05, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=100, n_jobs=-1, num_leaves=31,
              objective='regression', random_state=None, reg_alpha=0.0,
              reg_lambda=0.0, silent=True, subsample=1.0,
              subsample_for_bin=200000, subsample_freq=0, verbose=0)

In [98]:
preds_xgb = xg_reg.predict(X_val)
preds_rf = rf.predict(X_val)
preds_bag = bagging.predict(X_val)
preds_lgbm = gbm.predict(X_val)
test_preds_xgb = xg_reg.predict(X_test)
test_preds_rf = rf.predict(X_test)
test_preds_bag = bagging.predict(X_test)
test_preds_lgbm = gbm.predict(X_test)

In [99]:
X_val['pred_xgb'] = preds_xgb
X_val['pred_rf'] = preds_rf
X_val['pred_bag'] = preds_bag
X_val['pred_lgbm'] = preds_lgbm
X_test['pred_xgb'] = test_preds_xgb
X_test['pred_rf'] = test_preds_rf
X_test['pred_bag'] = test_preds_bag
X_test['pred_lgbm'] = test_preds_lgbm
regr.fit(X_val, y_val)
preds = regr.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

/home/gabriel/workspace/7506-1c2019/env/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/gabriel/workspace/7506-1c2019/env/lib/python3.5/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/gabriel/workspace/7506-1c2019/env/lib/python3.5/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_ind

RMSE: 68464.754986


#### Ventana 3

In [100]:
X_test, y_test = installsCheck3.iloc[:,:-1], installsCheck3.iloc[:,-1]
X, y = installs3.iloc[:,:-1], installs3.iloc[:,-1]
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3)
xg_reg.fit(X_train, y_train)
rf.fit(X_train, y_train)
bagging.fit(X_train, y_train)
gbm.fit(X_train, y_train, eval_set=[(X_val, y_val)], eval_metric='l1', early_stopping_rounds=15)

/home/gabriel/workspace/7506-1c2019/env/lib/python3.5/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[1]	valid_0's l2: 5.87049e+09	valid_0's l1: 66982.9
Training until validation scores don't improve for 15 rounds.
[2]	valid_0's l2: 5.8676e+09	valid_0's l1: 66962
[3]	valid_0's l2: 5.86689e+09	valid_0's l1: 66955.4
[4]	valid_0's l2: 5.86368e+09	valid_0's l1: 66931.7
[5]	valid_0's l2: 5.86095e+09	valid_0's l1: 66910.8
[6]	valid_0's l2: 5.85882e+09	valid_0's l1: 66895.7
[7]	valid_0's l2: 5.85695e+09	valid_0's l1: 66881
[8]	valid_0's l2: 5.85563e+09	valid_0's l1: 66870.4
[9]	valid_0's l2: 5.85407e+09	valid_0's l1: 66857.2
[10]	valid_0's l2: 5.85216e+09	valid_0's l1: 66842.1
[11]	valid_0's l2: 5.85065e+09	valid_0's l1: 66828.5
[12]	valid_0's l2: 5.84945e+09	valid_0's l1: 66817.1
[13]	valid_0's l2: 5.84873e+09	valid_0's l1: 66809.7
[14]	valid_0's l2: 5.84778e+09	valid_0's l1: 66799.7
[15]	valid_0's l2: 5.84704e+09	valid_0's l1: 66790.9
[16]	valid_0's l2: 5.84631e+09	valid_0's l1: 66784.4
[17]	valid_0's l2: 5.84602e+09	valid_0's l1: 66781.4
[18]	valid_0's l2: 5.84527e+09	valid_0's l1: 66774.

LGBMRegressor(bagging_fraction=0.8, bagging_freq=5, boosting_type='gbdt',
              class_weight=None, colsample_bytree=1.0, feature_fraction=0.9,
              importance_type='split', learning_rate=0.05, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=100, n_jobs=-1, num_leaves=31,
              objective='regression', random_state=None, reg_alpha=0.0,
              reg_lambda=0.0, silent=True, subsample=1.0,
              subsample_for_bin=200000, subsample_freq=0, verbose=0)

In [101]:
preds_xgb = xg_reg.predict(X_val)
preds_rf = rf.predict(X_val)
preds_bag = bagging.predict(X_val)
preds_lgbm = gbm.predict(X_val)
test_preds_xgb = xg_reg.predict(X_test)
test_preds_rf = rf.predict(X_test)
test_preds_bag = bagging.predict(X_test)
test_preds_lgbm = gbm.predict(X_test)
X_val['pred_xgb'] = preds_xgb
X_val['pred_rf'] = preds_rf
X_val['pred_bag'] = preds_bag
X_val['pred_lgbm'] = preds_lgbm
X_test['pred_xgb'] = test_preds_xgb
X_test['pred_rf'] = test_preds_rf
X_test['pred_bag'] = test_preds_bag
X_test['pred_lgbm'] = test_preds_lgbm
regr.fit(X_val, y_val)
preds = regr.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

/home/gabriel/workspace/7506-1c2019/env/lib/python3.5/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/home/gabriel/workspace/7506-1c2019/env/lib/python3.5/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
/home/gabriel/workspace/7506-1c2019/env/lib/python3.5/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Tr

RMSE: 69333.615342


#### Ventana 4

In [102]:
X_test, y_test = installsCheck4.iloc[:,:-1], installsCheck4.iloc[:,-1]
X, y = installs4.iloc[:,:-1], installs4.iloc[:,-1]
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3)
xg_reg.fit(X_train, y_train)
rf.fit(X_train, y_train)
bagging.fit(X_train, y_train)
gbm.fit(X_train, y_train, eval_set=[(X_val, y_val)], eval_metric='l1', early_stopping_rounds=15)

/home/gabriel/workspace/7506-1c2019/env/lib/python3.5/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[1]	valid_0's l2: 5.8186e+09	valid_0's l1: 66587.7
Training until validation scores don't improve for 15 rounds.
[2]	valid_0's l2: 5.814e+09	valid_0's l1: 66557.9
[3]	valid_0's l2: 5.81261e+09	valid_0's l1: 66548.1
[4]	valid_0's l2: 5.80936e+09	valid_0's l1: 66523.7
[5]	valid_0's l2: 5.8067e+09	valid_0's l1: 66502.1
[6]	valid_0's l2: 5.8038e+09	valid_0's l1: 66480.7
[7]	valid_0's l2: 5.80171e+09	valid_0's l1: 66463.3
[8]	valid_0's l2: 5.80005e+09	valid_0's l1: 66449.7
[9]	valid_0's l2: 5.79789e+09	valid_0's l1: 66431.5
[10]	valid_0's l2: 5.79676e+09	valid_0's l1: 66420.4
[11]	valid_0's l2: 5.79533e+09	valid_0's l1: 66406.8
[12]	valid_0's l2: 5.79302e+09	valid_0's l1: 66387.5
[13]	valid_0's l2: 5.79227e+09	valid_0's l1: 66378.3
[14]	valid_0's l2: 5.79187e+09	valid_0's l1: 66371.9
[15]	valid_0's l2: 5.79072e+09	valid_0's l1: 66359.2
[16]	valid_0's l2: 5.78982e+09	valid_0's l1: 66348.4
[17]	valid_0's l2: 5.78821e+09	valid_0's l1: 66332.4
[18]	valid_0's l2: 5.78577e+09	valid_0's l1: 66310.

LGBMRegressor(bagging_fraction=0.8, bagging_freq=5, boosting_type='gbdt',
              class_weight=None, colsample_bytree=1.0, feature_fraction=0.9,
              importance_type='split', learning_rate=0.05, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=100, n_jobs=-1, num_leaves=31,
              objective='regression', random_state=None, reg_alpha=0.0,
              reg_lambda=0.0, silent=True, subsample=1.0,
              subsample_for_bin=200000, subsample_freq=0, verbose=0)

In [103]:
preds_xgb = xg_reg.predict(X_val)
preds_rf = rf.predict(X_val)
preds_bag = bagging.predict(X_val)
preds_lgbm = gbm.predict(X_val)
test_preds_xgb = xg_reg.predict(X_test)
test_preds_rf = rf.predict(X_test)
test_preds_bag = bagging.predict(X_test)
test_preds_lgbm = gbm.predict(X_test)
X_val['pred_xgb'] = preds_xgb
X_val['pred_rf'] = preds_rf
X_val['pred_bag'] = preds_bag
X_val['pred_lgbm'] = preds_lgbm
X_test['pred_xgb'] = test_preds_xgb
X_test['pred_rf'] = test_preds_rf
X_test['pred_bag'] = test_preds_bag
X_test['pred_lgbm'] = test_preds_lgbm
regr.fit(X_val, y_val)
preds = regr.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

/home/gabriel/workspace/7506-1c2019/env/lib/python3.5/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/home/gabriel/workspace/7506-1c2019/env/lib/python3.5/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
/home/gabriel/workspace/7506-1c2019/env/lib/python3.5/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Tr

RMSE: 68245.998870


#### Ventana 5

In [104]:
X_test, y_test = installsCheck5.iloc[:,:-1], installsCheck5.iloc[:,-1]
X, y = installs5.iloc[:,:-1], installs5.iloc[:,-1]
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3)
xg_reg.fit(X_train, y_train)
rf.fit(X_train, y_train)
bagging.fit(X_train, y_train)
gbm.fit(X_train, y_train, eval_set=[(X_val, y_val)], eval_metric='l1', early_stopping_rounds=15)

/home/gabriel/workspace/7506-1c2019/env/lib/python3.5/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[1]	valid_0's l2: 5.78782e+09	valid_0's l1: 66412.2
Training until validation scores don't improve for 15 rounds.
[2]	valid_0's l2: 5.78473e+09	valid_0's l1: 66388.3
[3]	valid_0's l2: 5.78307e+09	valid_0's l1: 66377.6
[4]	valid_0's l2: 5.77993e+09	valid_0's l1: 66352.8
[5]	valid_0's l2: 5.77743e+09	valid_0's l1: 66331
[6]	valid_0's l2: 5.77334e+09	valid_0's l1: 66302.4
[7]	valid_0's l2: 5.76919e+09	valid_0's l1: 66274.9
[8]	valid_0's l2: 5.76671e+09	valid_0's l1: 66255.7
[9]	valid_0's l2: 5.76503e+09	valid_0's l1: 66241.3
[10]	valid_0's l2: 5.76196e+09	valid_0's l1: 66219
[11]	valid_0's l2: 5.75999e+09	valid_0's l1: 66203.8
[12]	valid_0's l2: 5.75789e+09	valid_0's l1: 66186.8
[13]	valid_0's l2: 5.75676e+09	valid_0's l1: 66175.8
[14]	valid_0's l2: 5.75453e+09	valid_0's l1: 66159.3
[15]	valid_0's l2: 5.75378e+09	valid_0's l1: 66150.6
[16]	valid_0's l2: 5.7531e+09	valid_0's l1: 66141.8
[17]	valid_0's l2: 5.75227e+09	valid_0's l1: 66132
[18]	valid_0's l2: 5.75171e+09	valid_0's l1: 66124
[1

LGBMRegressor(bagging_fraction=0.8, bagging_freq=5, boosting_type='gbdt',
              class_weight=None, colsample_bytree=1.0, feature_fraction=0.9,
              importance_type='split', learning_rate=0.05, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=100, n_jobs=-1, num_leaves=31,
              objective='regression', random_state=None, reg_alpha=0.0,
              reg_lambda=0.0, silent=True, subsample=1.0,
              subsample_for_bin=200000, subsample_freq=0, verbose=0)

In [105]:
preds_xgb = xg_reg.predict(X_val)
preds_rf = rf.predict(X_val)
preds_bag = bagging.predict(X_val)
preds_lgbm = gbm.predict(X_val)
test_preds_xgb = xg_reg.predict(X_test)
test_preds_rf = rf.predict(X_test)
test_preds_bag = bagging.predict(X_test)
test_preds_lgbm = gbm.predict(X_test)
X_val['pred_xgb'] = preds_xgb
X_val['pred_rf'] = preds_rf
X_val['pred_bag'] = preds_bag
X_val['pred_lgbm'] = preds_lgbm
X_test['pred_xgb'] = test_preds_xgb
X_test['pred_rf'] = test_preds_rf
X_test['pred_bag'] = test_preds_bag
X_test['pred_lgbm'] = test_preds_lgbm
regr.fit(X_val, y_val)
preds = regr.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

/home/gabriel/workspace/7506-1c2019/env/lib/python3.5/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/home/gabriel/workspace/7506-1c2019/env/lib/python3.5/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
/home/gabriel/workspace/7506-1c2019/env/lib/python3.5/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Tr

RMSE: 66503.355912


In [106]:
test = targets_sc_with_installs[['ref_type', 'ref_hash', 'total_apps', '%implicit', '%attributed', 'most_freq_lang', 'most_freq_app', 'model', 'ip', 'events_%wifi', 'event_apps', 'distinct_events', 'total_events']]

In [107]:
preds_xgb = xg_reg.predict(test)
preds_rf = rf.predict(test)
preds_bag = bagging.predict(test)
preds_lgbm = gbm.predict(test)

In [108]:
test['preds_xgb'] = preds_xgb
test['preds_rf'] = preds_rf
test['preds_bag'] = preds_bag
test['preds_lgbm'] = preds_lgbm

/home/gabriel/workspace/7506-1c2019/env/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/gabriel/workspace/7506-1c2019/env/lib/python3.5/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/gabriel/workspace/7506-1c2019/env/lib/python3.5/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_ind

In [109]:
predictions_sc = regr.predict(test)
predictions_sc

array([126011.46715009, 132295.59928197, 125123.47186122, ...,
       129549.25754797, 132295.59928197, 132295.59928197])

In [112]:
targets_sc['obj'] = predictions_sc
predicciones_target_sc = target[target['ref_hash'].str.contains('_sc')]
predicciones_target_sc['obj'] = predictions_sc
target.update(predicciones_target_sc)
# sc_pred = pd.read_csv('../datos/PrediccionesKNN-RF-XGBoost.txt', header = None)
# sc_pred.index = np.arange(1, 8074, 2)
targets_st['obj'] = predictions_sc[0]
target.update(targets_st)

/home/gabriel/workspace/7506-1c2019/env/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/gabriel/workspace/7506-1c2019/env/lib/python3.5/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/gabriel/workspace/7506-1c2019/env/lib/python3.5/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to b

In [113]:
target.to_csv('../datos/preds-blending.csv', index = False)